# tune vl的模型，全能的

In [1]:
import numpy as np
import pandas as pd
base_dir = '/data/coding/patent_qa/train/'
df_question = pd.read_json("/data/coding/patent_qa/train/questions.jsonl",lines=True)
quesion_vector = np.load('all_train_question_vectors.npy')
train_ocr_page_num_mapping = pd.read_csv('train_ocr_page_num_mapping.csv')
train_ocr_vectors = np.load('train_ocr_vectors.npy')
train_pdf_image_vectors = np.load("train_pdf_img_vectors.npy") # 前400个
train_pdf_image_page_num_mapping = pd.read_csv('train_pdf_img_page_num_mapping.csv') # 前400个

In [4]:


# 给定了图片的情况下，有图片本身，也需要召回对应的文本
def get_similar_text_embedding(base_dir,document_name,question_idx,top_k):
    document_name = df_question.document[question_idx].split('.')[0]
    vec_idx = train_ocr_page_num_mapping[train_ocr_page_num_mapping['file_name']==document_name]['index'].values
    candidate_vec = train_ocr_vectors[vec_idx]
    query_vec = quesion_vector[question_idx]
    cos_sim = np.dot(candidate_vec, query_vec) / (np.linalg.norm(candidate_vec) * np.linalg.norm(query_vec))
    # 获取最相似的top_k个索引
    top_k_indices = np.argsort(cos_sim)[-top_k:][::-1]
    retrived_idx = vec_idx[top_k_indices] # 最相近的top_k个
    retrived_page_num = train_ocr_page_num_mapping.loc[retrived_idx]['page_num'].to_list()
    text_list = []
    for i in range(len(retrived_page_num)):
        text_file = base_dir + '/pdf_ocr/' + document_name + '/' + str(retrived_page_num[i]) +'.txt'
        with open(text_file,'r',encoding='utf-8') as f:
            text_list.append(f.read())
    return text_list # 返回一个list,大小最大为top_k


def get_similar_image_embedding(base_dir,document_name,question_idx,top_k):
    document_name = df_question.document[question_idx].split('.')[0]
    vec_idx = train_pdf_image_page_num_mapping[train_pdf_image_page_num_mapping['file_name']==document_name]['index'].values
    candidate_vec = train_pdf_image_vectors[vec_idx]
    query_vec = quesion_vector[question_idx]
    cos_sim = np.dot(candidate_vec, query_vec) / (np.linalg.norm(candidate_vec) * np.linalg.norm(query_vec))
    # 获取最相似的top_k个索引
    top_k_indices = np.argsort(cos_sim)[-top_k:][::-1]
    retrived_idx = vec_idx[top_k_indices] # 最相近的top_k个
    retrived_page_num = train_pdf_image_page_num_mapping.loc[retrived_idx]['page_num'].to_list()
    image_list = []
    for i in range(len(retrived_page_num)):
        image_file = base_dir + '/pdf_img/' + document_name + '/' + str(retrived_page_num[i]) +'.jpg'
        image_list.append(image_file)
    return image_list # 返回一个list,大小最大为top_k

def get_text_answer_vl(document_name,question,options,quesion_idx):
    prompt ="你是一个专利内容分析专家，请根据我提供的专利内容回答我的问题。\n"
    prompt += "【我的问题】【"
    prompt += (question +"】\n")
    prompt += "【选项】【"
    prompt += (' '.join(options) + "】\n")
    prompt += ("专利内容为：\n")
    retrived_list = get_similar_text_embedding(base_dir,document_name,quesion_idx,2)
    prompt += '\n'.join(retrived_list)
    prompt += ("\n\n请你分析专利内容后，回答我的单选题，回答正确选项字母，你的答案为：\n")
    #sampling_params = SamplingParams(temperature=0)
    #outputs = text_qa_llm.generate(question,sampling_params)
    #return outputs[0].outputs[0].text
    messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                ],
            }
        ]
    query = prompt
    # data = dict()
    # data['messages'] = messages
    # infer_requests = [InferRequest(**data)]
    # response = infer_batch(engine, infer_requests)
    return query,[]

def get_image_answer(document_name,question,options,question_idx):
    question1 ="你是一个专利内容分析专家，请根据我提供的专利内容回答我的单选题。\n"
    question1 += "【我的问题】【"
    question1 += (question +"】\n")
    question1 += "【选项】【"
    question1 += (' '.join(options) + "】\n")
    question1 += ("专利内容为：\n")
    retrived_list = get_similar_image_embedding(base_dir,document_name,question_idx,2)
    question2 = ("\n\n请你分析专利内容后，回答我的单选题，直接回答正确选项字母，你的答案为：\n")
    if len(retrived_list)>1:
        query = ''
        query += question1
        query += '<image>'
        query += '<image>'
        query += question2
        images = []
        images.append(retrived_list[0])
        images.append(retrived_list[1])
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question1},
                    {
                        "type": "image",
                        "image": retrived_list[0],
                    },
                    {
                        "type": "image",
                        "image": retrived_list[1],
                    },
                    {"type": "text", "text": question2},
                ],
            }
        ]
    else:
        query = ''
        query += question1
        query += '<image>'
        query += question2
        images = []
        images.append(retrived_list[0])
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question1},
                    {
                        "type": "image",
                        "image": retrived_list[0],
                    },
                    {"type": "text", "text": question2},
                ],
            }
        ]
    # data = dict()
    # data['messages'] = messages
    # infer_requests = [InferRequest(**data)]
    # response = infer_batch(engine, infer_requests)
    return query,images

def get_mix_answer(document_name,pic_page_num,question,options,question_idx):
    question1 ="你是一个专利内容分析专家，请根据我提供的专利内容回答我的问题。\n"
    question1 += "【我的问题】【"
    question1 += (question +"】\n")
    question1 += "【选项】【"
    question1 += (' '.join(options) + "】\n")
    question1 += ("专利内容为：\n")
    retrived_list = get_similar_text_embedding(base_dir,document_name,question_idx,2)
    question2 = ("\n\n请你分析专利内容后，回答我的单选题，直接回答正确选项字母，你的答案为：\n")
    messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question1},
                    {
                        "type": "image",
                        "image": base_dir + '/pdf_img/' + document_name.split('.')[0]+ '/' + str(pic_page_num) +'.jpg',
                    },
                    {"type": "text", "text": '\n'.join(retrived_list)},
                    {"type": "text", "text": question2},
                ],
            }
    ]
    query = ''
    query += question1
    query += '<image>'
    query += '\n'.join(retrived_list) 
    query += question2
    images = [base_dir + '/pdf_img/' + document_name.split('.')[0]+ '/' + str(pic_page_num) +'.jpg']
    # data = dict()
    # data['messages'] = messages
    # infer_requests = [InferRequest(**data)]
    # response = infer_batch(engine, infer_requests)
    return query, images


def get_mix_answer_img(document_name,pic_page_num,question,options,question_idx):
    question1 ="你是一个专利内容分析专家，请根据我提供的专利内容回答我的问题。\n"
    question1 += "【我的问题】【"
    question1 += (question +"】\n")
    question1 += "【选项】【"
    question1 += (' '.join(options) + "】\n")
    question1 += ("专利内容为：\n")
    #retrived_list = get_similar_text_embedding(base_dir,document_name,question_idx,2)
    retrived_list = get_similar_image_embedding(base_dir,document_name,question_idx,2)
    question2 = ("\n\n请你分析专利内容后，回答我的单选题，直接回答正确选项字母，你的答案为：\n")
    if len(retrived_list)>1:
        query = ''
        query += question1
        query += '<image>'
        query += '<image>'
        query += '<image>'
        query += question2
        images = []
        images.append(base_dir + '/pdf_img/' + document_name.split('.')[0]+ '/' + str(pic_page_num) +'.jpg')
        images.append(retrived_list[0])
        images.append(retrived_list[1])
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question1},
                    {
                        "type": "image",
                        "image": base_dir + '/pdf_img/' + document_name.split('.')[0]+ '/' + str(pic_page_num) +'.jpg',
                    },
                    {
                        "type": "image",
                        "image": retrived_list[0],
                    },
                    {
                        "type": "image",
                        "image": retrived_list[1],
                    },
                    {"type": "text", "text": question2},
                ],
            }
        ]
    else:
        query = ''
        query += question1
        query += '<image>'
        query += question2
        images = []
        images.append(base_dir + '/pdf_img/' + document_name.split('.')[0]+ '/' + str(pic_page_num) +'.jpg')
        images.append(retrived_list[0])
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question1},
                    {
                        "type": "image",
                        "image": base_dir + '/pdf_img/' + document_name.split('.')[0]+ '/' + str(pic_page_num) +'.jpg',
                    },
                    {
                        "type": "image",
                        "image": retrived_list[0],
                    },
                    {"type": "text", "text": question2},
                ],
            }
        ]

    return query, images



In [7]:
# def write_to_jsonl_text_train_set(x):
#     tmp_dict = dict()
#     tmp_dict['query'] = x['query']
#     tmp_dict['document'] = x['document']
#     tmp_dict['question_idx'] = int(x['question_idx'])
#     tmp_dict['response'] = x['response']
#     with open('train_dataset_for_text.jsonl', 'a', encoding='utf-8') as f:
#         f.write(json.dumps(tmp_dict, ensure_ascii=False) + '\n')

# def write_to_jsonl_img_train_set(x):
#     tmp_dict = dict()
#     tmp_dict['query'] = x['query']
#     tmp_dict['document'] = x['document']
#     tmp_dict['question_idx'] = int(x['question_idx'])
#     tmp_dict['response'] = x['response']
#     tmp_dict['images'] = x['images']
#     with open('train_dataset_for_image.jsonl', 'a', encoding='utf-8') as f:
#         f.write(json.dumps(tmp_dict, ensure_ascii=False) + '\n')


import re
from tqdm import trange
import json
for i in trange(len(df_question)):
    question = df_question.loc[i,'question']
    document_name = df_question.loc[i,'document']
    options = df_question.loc[i,'options']
    true_answer = df_question.loc[i,'answer']
    full_question = question + ' '.join(options)
    answer = ''
    text_answer = ''
    image_answer = ''
    if "第" in question and "页" in question and "图": # 问题含有图片
        pic_page_num = re.findall(r"第(\d+)页", question)[0]
        pic_page_num = int(pic_page_num)
        query,images = get_mix_answer(document_name,pic_page_num,question,options,i)
        result_dict = dict()
        result_dict['question_idx'] = i
        result_dict['document'] = document_name
        result_dict['query'] = query
        result_dict['response'] = true_answer
        result_dict['images'] = images
        #write_to_jsonl_img_train_set(result_dict)
        query,images = get_mix_answer_img(document_name,pic_page_num,question,options,i)
        result_dict = dict()
        result_dict['question_idx'] = i
        result_dict['document'] = document_name
        result_dict['query'] = query
        result_dict['response'] = true_answer
        result_dict['images'] = images
        #write_to_jsonl_img_train_set(result_dict)
    else:
        query,images = get_text_answer_vl(document_name,question,options,i) # 使用文本来回答
        result_dict = dict()
        result_dict['question_idx'] = i
        result_dict['document'] = document_name
        result_dict['query'] = query
        result_dict['response'] = true_answer
        #write_to_jsonl_text_train_set(result_dict)
        query,images = get_image_answer(document_name,question,options,i) # 使用图像来回答
        result_dict = dict()
        result_dict['question_idx'] = i
        result_dict['document'] = document_name
        result_dict['query'] = query
        result_dict['response'] = true_answer
        result_dict['images'] = images
        #write_to_jsonl_img_train_set(result_dict)
    break



  0%|          | 0/300 [00:00<?, ?it/s]


In [11]:
print(document_name)
print(question)
print(options)

CN213444549U.pdf
根据专利文本，以下哪个是该货物靠边规整处理机构的主要功能？
['A. 实现对货物的加热处理。', 'B. 实现对货物的靠边规整处理。', 'C. 实现对货物的分拣功能。', 'D. 实现对货物的称重功能。']


In [10]:
options

['A. 实现对货物的加热处理。', 'B. 实现对货物的靠边规整处理。', 'C. 实现对货物的分拣功能。', 'D. 实现对货物的称重功能。']

In [14]:
query,images = get_text_answer_vl(document_name,question,options,i)
print(query)

你是一个专利内容分析专家，请根据我提供的专利内容回答我的问题。
【我的问题】【根据专利文本，以下哪个是该货物靠边规整处理机构的主要功能？】
【选项】【A. 实现对货物的加热处理。 B. 实现对货物的靠边规整处理。 C. 实现对货物的分拣功能。 D. 实现对货物的称重功能。】
专利内容为：
CN 213444549 U 说明书 1/2 页

一种用于辊筒输送机的货物靠边规整处理机构

技术领域
[0001] 本实用新型涉及滚筒输送技术领域，特别涉及一种用于辊筒输送机的货物靠边规整处理机构。

背景技术
[0002] 辊筒输送机适用于底部是平面的物品输送，主要由传动滚筒、机架、支架、驱动部等部分组成。具有输送量大，速度快，运转轻快，能够实现多品种共线分流输送的特点。辊筒输送机之间易于衔接过滤，可用多条滚筒线及其它输送设备或专机组成复杂的物流输送系统。
[0003] 然而，在实际应用过程中，有些物品在输送至末端时，需要集中或规整处理；而且，现有技术中，缺少对辊筒输送机上物品的自动规整处理功能。

实用新型内容
[0004] 本实用新型要解决的技术问题是提供一种用于辊筒输送机的货物靠边规整处理机构，该货物靠边规整处理机构便于对辊筒输送机上的货物进行靠边规整处理。
[0005] 为了解决上述技术问题，本实用新型的方案为：
[0006] 一种用于辊筒输送机的货物靠边规整处理机构，包括平行设置的两块侧板，所述侧板之间朝同一方向倾斜设置有若干转辊，所述转辊两端分别通过转轴连接在两侧板之间，所述转辊上配置有皮带槽，所述皮带槽内配置有联动皮带，相邻所述转辊之间通过所述联动皮带连接，所述侧板的上设置有驱动装置，所述驱动装置与所述转辊匹配连接。
[0007] 所述侧板上设置有栏板。
[0008] 所述转辊的倾斜角度为5-10°。
[0009] 所述转轴通过轴承与所述侧板连接。
[0010] 所述货物靠边规整处理机构两端设置有支架，且所述支架与所述货物靠边规整处理机构两端的转辊转动连接。
[0011] 所述货物靠边规整处理机构的入口端与滚筒输送机连接，出口端连接有下游程序设备。
[0012] 与现有技术相比，本实用新型的有益效果为：
[0013] 本申请在应用过程中，物料从入口端进入转辊上，由于转辊倾斜设置，从而使其上的物品向本申请货物靠边规整处理机构的一个侧边移动，当移动到侧边不能再移动时